In [3]:
# !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu123

In [1]:
# using Kernel PyTorch-1.10
# using Kernel PyTorch-1.10
# using Kernel PyTorch-1.10

from gpn.data import GenomeMSA #, Tokenizer
import gpn.model

# from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from transformers import AutoModel #, AutoModelForMaskedLM
import torch
import numpy as np
import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# import pickle
import re
import os
import csv
import warnings
warnings.filterwarnings('ignore')

%run preprocess_utility.py

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
model_path = "songlab/gpn-msa-sapiens"
# msa_path = "zip:///::89.zarr.zip"
msa_path = "zip:///::/home/sunhuaikuan/ondemand/blue_gpn/examples/msa/89.zarr.zip"
genome_msa = GenomeMSA(msa_path)
model = AutoModel.from_pretrained(model_path).to(device)
model.eval();

Loading MSA...
Loading MSA... Done


Some weights of the model checkpoint at songlab/gpn-msa-sapiens were not used when initializing GPNRoFormerModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing GPNRoFormerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPNRoFormerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Main Function to get Embedding

In [3]:
comp = {'A':1, 'C':2, 'G':3, 'T':4}

max_seqlen=128

def Genosome2Embedding(chrom, pos_start, pos_end, y):  #  ref,alt,
    # tokenized msa
    # pos_start = pos - 65
    # pos_end=pos_start + 128
    msa = genome_msa.get_msa(str(chrom), pos_start, pos_end, strand="+", tokenize=True)
    # print(msa.shape)

    msa = torch.tensor(np.expand_dims(msa, 0).astype(np.int64))
    # msa

    # separating human from rest of species
    input_ids, aux_features = msa[:, :, 0], msa[:, :, 1:]
    
    input_ids = input_ids.to(device)
    aux_features = aux_features.to(device)


    with torch.no_grad():
        last_hidden_state = model(input_ids=input_ids, aux_features=aux_features).last_hidden_state
        
        # Mean Pooling: Compute the mean across the sequence length (dim=1)
        mean_pooled = last_hidden_state.mean(dim=1)  # Shape: (batch_size, embedding_dim)

    feature=np.append(mean_pooled.cpu().numpy(),  [ y])  # chrom,  pos_end-pos_start, comp[ref],comp[alt],
    
    return feature


### Output CSV File

In [4]:
def output2CSV(df, csv_Filename):

    if os.path.exists(csv_Filename):
        os.remove(csv_Filename)

    rows=[]
    # progress_bar = tqdm(total=df.shape[0], desc="Processing")
    for index, row in df.iterrows():
        
        chrom=row['CHROM']
        pos_start=row['START']
        pos_end=row['END']
        # ref=row['REF']
        # alt=row['ALT']        
        # info=row['INFO']
        y=row['y']
        try:
            embedding  =  Genosome2Embedding(chrom, pos_start,pos_end,y) # ref,alt, 
            rows.append(embedding)
    
        except Exception as e:
            print(f"exception caught: {e}"+str(row['CHROM'])+'-'+str(row['START']))
    
        # progress_bar.update(1)


        if ((index % 5000) ==0):
            with open(csv_Filename, mode='a', newline='') as file:
                writer = csv.writer(file)
                # writer.writerow(rows)
                for row in rows:
                    writer.writerow(row)
            rows=[]
            # progress_bar.update(1)
            print(f"complete index={index}")

    
    with open(csv_Filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        # writer.writerow(rows)
        for row in rows:
            writer.writerow(row)

    print(f"Create File: "+csv_Filename)
    # progress_bar.reset()
    # progress_bar.close()

In [5]:
### PreProcess Dataset
# def PreProcess(data_filename):
#     # df=pd.read_csv(data_filename, delimiter='\t',  names=['CHROM', 'POS', 'REF','ALT','INFO'])  # No head column row
#     df=pd.read_csv(data_filename, header=0, sep='\t', index_col=None)   # With head column row

#     df=df[(df['REF'].str.len() == 1) & (df['ALT'].str.len() == 1)]
#     df['CHROM'] = df['CHROM'].str.replace('chr', '')
#     df['y']=0
#     return df

# df1=PreProcess('gnomad.v4.1.exon.txt')
# df1
# output2CSV(df1,'./gnomad.v4.1.exon.gpn.csv')

### Load mathylation data

In [6]:
datafile='methylation'

import gzip
import pandas as pd

max_length= 128 # 186


# Open the gzipped file and read it into a DataFrame
data_filename = '/home/sunhuaikuan/ondemand/blue_papers/DNA_LLM_REVIEW/datasets/task05-methylation/GSM6637962_CpG_coverage20_GRCh38.bed.gz'  
with gzip.open(data_filename, 'rt') as f:
    df = pd.read_csv(f, sep='\t')  # Automatically detects header from the file


df['CHROM'] = df['CHROM'].str.replace('chr', '', regex=False)

df['START']=df['FROM']- max_length //2 -1

df['END']=df['START'] + max_length


# df['REF']='A'
# df['ALT']='A'
# df['SIZE']=max_length

df = df.rename(columns={'Percentage':'y'})
df = df.drop(['FROM','TO','Coverage'], axis=1)


cols = df.columns.tolist()

# Move the 3rd column to the last position
cols.append(cols.pop(2))

# Reorder the DataFrame
df = df[cols]
df=df[~df['CHROM'].str.contains('KI',na=False)]
df=df[~df['CHROM'].str.contains('GL',na=False)]
df=df[~df['CHROM'].str.contains('M',na=False)]
df


,CHROM,y,END,START
0,1,100,788470,788342
1,1,80,788535,788407
2,1,95,788888,788760
3,1,90,788907,788779
4,1,100,788922,788794
...,...,...,...,...
54789,Y,65,56870966,56870838
54790,Y,17,56871046,56870918
54791,Y,34,56871172,56871044
54792,Y,0,56871565,56871437


In [7]:
# df['CHROM'].value_counts()

In [8]:
# max_seqlen=128


# # datafile='Homo_sapiens.GRCh38.109'
# # datafile='clinvar_20240805.10missense'
# # datafile='clinvar_20240805.missense_matched'
# datafile='clinvar_20240805.noncoding'


# df=pd.read_csv(datafile+'.txt', delimiter='\t')

# columns_to_keep=['CHROM','POS','ID','REF','ALT','Pathogenicity']
# df = df[columns_to_keep]

# # Merge CHROM=9 and '9' etc
# for i in range(1,23):
#     df.loc[df['CHROM']==i,'CHROM']=str(i)
# # df['CHROM'].value_counts()

# df['START']=df['POS']- max_seqlen //2 -1

# df['END']=df['START'] + max_seqlen

# df=df[~df['CHROM'].isna()]

# Pathogenicity_dict={'B':0,'P':1}
# df['y'] = df['Pathogenicity'].map(Pathogenicity_dict)
# # df['CHROM'].value_counts()
# df

In [9]:
%%time

output2CSV(df,datafile+'_gpn_embedding.csv')

complete index=0
complete index=5000
complete index=10000
complete index=15000
complete index=20000
complete index=25000
complete index=30000
complete index=35000
complete index=40000
complete index=45000
Create File: methylation_gpn_embedding.csv
CPU times: user 9min 18s, sys: 16.7 s, total: 9min 35s
Wall time: 14min 12s


### Load CSV File

In [10]:
# def load_embedding_file(csv_filename):

#     df=pd.read_csv(csv_filename)
    
#     column_names = [f'{i}' for i in range(1, df.shape[1]-2)]
#     column_names.extend(['CHROM', 'SIZE',  'y'])  # 'REF', 'ALT', 
    
#     df.columns = column_names
#     return df

df = load_embedding_file(datafile+'_gpn_embedding.csv')
df

,1,2,3,4,5,6,7,8,9,10,...,760,761,762,763,764,765,766,767,768,y
0,0.612933,0.057651,-1.246180,0.277122,-0.913016,-0.190393,-0.607857,0.931990,0.123141,-0.208136,...,-1.309597,0.027291,0.728741,0.025583,-0.226388,-0.122658,-0.830180,0.287627,0.697681,80.0
1,0.439794,-0.217588,-1.365778,0.898174,-0.855323,-0.057502,-0.541308,0.825290,-0.152932,0.057166,...,-1.478570,-0.206125,0.665346,0.037774,-0.350043,-0.394456,-0.936075,0.136769,0.755584,95.0
2,0.509560,-0.267472,-1.300921,0.996084,-0.885385,0.017741,-0.389727,0.736749,-0.024453,-0.015932,...,-1.499621,-0.270688,0.674775,0.030219,-0.439443,-0.394929,-0.962380,0.199156,0.799231,90.0
3,0.521640,-0.312444,-1.231837,0.999817,-0.902015,0.040940,-0.388363,0.678714,0.115234,-0.002658,...,-1.482253,-0.241800,0.587185,-0.061250,-0.514301,-0.447875,-0.878306,0.179583,0.790959,100.0
4,0.524937,-0.312998,-1.194757,1.013375,-0.899023,0.025699,-0.360354,0.680852,0.182384,-0.016140,...,-1.480661,-0.231768,0.515274,-0.073867,-0.556997,-0.451583,-0.834344,0.167359,0.820079,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47947,1.011808,-0.340859,-0.550905,0.540617,-0.895378,-0.361927,-0.386061,0.108030,0.178881,-0.575671,...,-1.276941,-0.365299,1.221117,0.361901,-0.237325,-0.175215,-1.330128,0.236117,0.664278,65.0
47948,0.920000,-0.485343,-0.189774,0.895718,-0.681114,-0.384050,-0.271887,-0.290705,0.370207,-0.471858,...,-1.058086,-0.546873,0.958808,0.289077,-0.237743,-0.465524,-1.134478,0.210529,0.553010,17.0
47949,0.988282,-0.167276,-0.147423,-0.331529,-0.631022,-0.981502,-0.874661,0.072641,0.650392,-0.628087,...,-0.723373,-0.058664,0.731405,0.305108,0.059723,0.031155,-1.170336,-0.064444,0.422516,34.0
47950,0.748591,-0.345141,-0.674358,0.946595,-0.722178,-0.245178,-0.144482,0.193256,0.020578,-0.512533,...,-1.154925,-0.550619,0.952435,0.460786,-0.215765,-0.452896,-1.396009,0.284754,0.886341,0.0
